## Check Monte Carlo Simulation
Check if we can afford to reduce the number of simulations to 100,00 and still get reliable results, to speed up the creation of the dataset for ML models

In [27]:
from dotenv import dotenv_values
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
from scipy import stats as st
from tqdm.notebook import tqdm
import time
from IPython.display import display

warnings.filterwarnings("ignore")
import sys
from pathlib import Path

# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0] / "src")


if module_path not in sys.path:
    sys.path.append(module_path)
    
from MMAR.MMAR import MMAR

### Load variables

In [2]:
start_date = "2005-01-01"
end_date = "2022-12-31"
df_path = "../data/raw/spy.csv"

In [3]:
config = dotenv_values("../config/.env")

In [4]:
ALPHA_API = config["ALPHA_API"]

In [5]:
def get_df(path: str) -> pd.DataFrame:
    if os.path.isfile(path):
        df = pd.read_csv(path, parse_dates=True, index_col=0)
    else:
        df = yf.download("SPY", start=start_date, end=end_date)
        df.to_csv(path)
    return df

### Get files

In [6]:
spy = get_df(df_path)

In [7]:
spy = spy[spy.index < "31-12-2017"]

### Simulation using various sizes and steps

In [9]:
mmar = MMAR(spy["Close"], volume=spy["Volume"])

In [30]:
S0 = spy.Close.iloc[-1]
steps = [21, 252, 504]
num_simulations = [100_000, 250_000, 500_000, 1_000_000]

In [31]:
results = {}
for n in steps:
    print(f"Steps: {n}")
    results[n] = {}
    for num_sim in num_simulations:
        start = time.time()
        simulation = mmar.get_MMAR_MC(S0, n=n, num_sim=num_sim)
        stop = time.time()
        price = simulation[:, -1:].mean()
        std = simulation[:, -1:].std()
        exec_time = stop-start
        print(f"Using {num_sim:,.0f} simlations and {n} steps, the final price is {price:.2f} with std: {std:.2f}, in {exec_time:.2f} seconds.")
        results[n][num_sim]={"Price":price, "Std":std, "Exec_Time":exec_time}
    print()

Steps: 21
Using 100,000 simlations and 21 steps, the final price is 267.10 with std: 11.61, in 0.27 seconds.
Using 250,000 simlations and 21 steps, the final price is 267.08 with std: 11.60, in 0.08 seconds.
Using 500,000 simlations and 21 steps, the final price is 267.13 with std: 11.60, in 0.29 seconds.
Using 1,000,000 simlations and 21 steps, the final price is 267.10 with std: 11.60, in 0.30 seconds.

Steps: 252
Using 100,000 simlations and 252 steps, the final price is 269.26 with std: 36.32, in 0.29 seconds.
Using 250,000 simlations and 252 steps, the final price is 269.28 with std: 36.54, in 0.89 seconds.
Using 500,000 simlations and 252 steps, the final price is 269.33 with std: 36.47, in 1.71 seconds.
Using 1,000,000 simlations and 252 steps, the final price is 269.31 with std: 36.45, in 3.39 seconds.

Steps: 504
Using 100,000 simlations and 504 steps, the final price is 272.70 with std: 58.71, in 0.65 seconds.
Using 250,000 simlations and 504 steps, the final price is 272.92 

In [49]:
for k in results.keys():
    print(f"Number of steps: {k}")
    display(pd.DataFrame.from_dict(results[k], orient="index")
    .assign(ci_high= lambda df: df.Price + df.Std/np.sqrt(df.index))
    .assign(ci_low= lambda df: df.Price - df.Std/np.sqrt(df.index))
    .round(decimals=2)
    )
    print()

Number of steps: 21


,Price,Std,Exec_Time,ci_high,ci_low
100000,267.10,11.61,0.27,267.13,267.06
250000,267.08,11.60,0.08,267.10,267.06
500000,267.13,11.60,0.29,267.14,267.11
1000000,267.10,11.60,0.30,267.11,267.09



Number of steps: 252


,Price,Std,Exec_Time,ci_high,ci_low
100000,269.26,36.32,0.29,269.38,269.15
250000,269.28,36.54,0.89,269.35,269.20
500000,269.33,36.47,1.71,269.38,269.28
1000000,269.31,36.45,3.39,269.35,269.27



Number of steps: 504


,Price,Std,Exec_Time,ci_high,ci_low
100000,272.70,58.71,0.65,272.88,272.51
250000,272.92,58.72,1.81,273.03,272.80
500000,273.02,58.66,3.71,273.11,272.94
1000000,273.05,58.63,8.14,273.11,272.99


Since we concentrate on the monthly time frame (21 days), we consider the error acceptable.